# LLM Requirements Agent Demo — Primary Loop Valve → Requirements Package

This notebook demonstrates an **LLM-assisted workflow** on top of the existing deterministic requirements engine in this repository.

## What you will do
1. **Locate the repo** (assumes this notebook lives in `repo_root/demo/`)
2. Load the **ValveProfile schema** and see what fields are required / enumerated
3. Create an **LLM-enabled agent** (or run in manual mode if no LLM configured)
4. Build a ValveProfile in one of two ways:
   - **A. Load an existing profile** (`demo/valve_profile_example.json`)
   - **B. Elicit a profile** by iteratively applying user messages (LLM-assisted) or filling required fields manually
5. Run the deterministic pipeline to generate:
   - `requirements_instance.json`
   - `validation_report.json` + `validation_report.csv`
   - `manifest.json`
   - a zipped package for hand-off to downstream agents

## LLM configuration (optional)
This repo supports **optional** LLM backends via environment variables (see `src/llm_client.py`).

- **Ollama**
  ```bash
  export LLM_MODE=ollama
  export OLLAMA_MODEL=llama3.1
  ```

- **OpenAI-compatible endpoint**
  ```bash
  export LLM_MODE=openai_compat
  export LLM_BASE_URL=http://localhost:8000
  export LLM_API_KEY=...
  export LLM_MODEL=...
  ```

If no LLM is configured, the notebook still works in **manual mode**.


## 1) Setup: find repo root and enable imports
This assumes the notebook is located in `repo_root/demo/`.
We add `repo_root/` to `sys.path` so imports like `from src.engine import ...` work.


In [ ]:

from pathlib import Path
import sys

CWD = Path.cwd().resolve()

def find_repo_root(start: Path) -> Path:
    for p in (start, *start.parents):
        if (p / "src" / "engine.py").exists() and (p / "schemas").exists() and (p / "data").exists():
            return p
    raise FileNotFoundError("Could not find repo root (expected src/engine.py, schemas/, data/).")

# If running from demo/, the repo root is typically the parent.
seed = CWD.parent if CWD.name.lower() == "demo" else CWD
REPO_ROOT = find_repo_root(seed)

DEMO_DIR = REPO_ROOT / "demo"
OUT_DIR = DEMO_DIR / "out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("CWD      :", CWD)
print("REPO_ROOT :", REPO_ROOT)
print("DEMO_DIR  :", DEMO_DIR)
print("OUT_DIR   :", OUT_DIR)
print("sys.path[0]:", sys.path[0])


## 2) Import the deterministic engine + the LLM add-on
If you have not installed the add-on files yet, you should have these new files under `src/`:
- `agent.py`, `profile_builder.py`, `llm_client.py`, `code_rag.py`, `chat_cli.py`

If this cell fails with `ImportError`, unzip/copy the add-on patch into the repo root and re-run.


In [ ]:

# Deterministic engine is always present in the repo
from src.engine import filter_and_instantiate

# LLM add-on (optional) – requires the add-on files to exist in src/
try:
    from src.agent import RequirementsChatAgent, AgentPaths
    from src.llm_client import from_env
    HAS_LLM_ADDON = True
except Exception as e:
    HAS_LLM_ADDON = False
    print("LLM add-on not available yet:", repr(e))

print("HAS_LLM_ADDON =", HAS_LLM_ADDON)


## 3) Load schema + library + an example profile
We load:
- the **ValveProfile schema** (`schemas/valve_profile.schema.json`)
- the **150-requirement template** (`data/primary_loop_valve_baseline_v0.2_150reqs.json`)
- an example profile from `demo/` (fallback to `examples/`)


In [ ]:

import json

schema_path = REPO_ROOT / "schemas" / "valve_profile.schema.json"
template_path = REPO_ROOT / "data" / "primary_loop_valve_baseline_v0.2_150reqs.json"

profile_path_candidates = [
    REPO_ROOT / "demo" / "valve_profile_example.json",
    REPO_ROOT / "examples" / "valve_profile_example.json",
]

profile_path = next((p for p in profile_path_candidates if p.exists()), None)
if profile_path is None:
    raise FileNotFoundError("Could not find valve_profile_example.json in demo/ or examples/.")

valve_profile_schema = json.loads(schema_path.read_text(encoding="utf-8"))
template = json.loads(template_path.read_text(encoding="utf-8"))
example_profile = json.loads(profile_path.read_text(encoding="utf-8"))

print("Schema:", schema_path)
print("Template:", template_path)
print("Example profile:", profile_path)
print("Template ID:", template.get("template_id"))


### 3.1 What fields are required? Which fields are enums?
We extract:
- `required` fields: must be present for a complete ValveProfile
- `enum` fields: constrained to a specific set of allowed values


In [ ]:

required_fields = valve_profile_schema.get("required", [])
props = valve_profile_schema.get("properties", {})
enum_fields = {k: v.get("enum", []) for k, v in props.items() if isinstance(v, dict) and "enum" in v}

print("Required fields (schema):")
print(required_fields)

print("\nEnum fields:")
for k, opts in sorted(enum_fields.items()):
    print(f"- {k}: {opts}")


## 4) Create an agent (LLM-enabled if configured)
If the add-on is installed, we create `RequirementsChatAgent`.

The agent can:
- maintain an evolving ValveProfile
- (optionally) use an LLM to extract profile updates from free text
- run the deterministic engine and package outputs
- answer developer questions grounded on this repo's code (code-RAG)


In [ ]:

if not HAS_LLM_ADDON:
    raise RuntimeError(
        "LLM add-on is not installed. Add the add-on files into src/ and re-run this notebook.\n"
        "Expected: src/agent.py, src/profile_builder.py, src/llm_client.py, src/code_rag.py"
    )

paths = AgentPaths(
    repo_root=REPO_ROOT,
    template_path=template_path,
    schema_path=schema_path,
    out_dir=OUT_DIR,
)

llm = from_env()  # may be None
agent = RequirementsChatAgent(paths=paths, llm=llm)

print("LLM configured?", bool(llm))
print("Missing required fields (fresh profile):", agent.missing_required())


## 5) Pretty display of the ValveProfile
The agent can render the current profile as a markdown table.
In a notebook, we can render this nicely using `IPython.display.Markdown`.


In [ ]:

from IPython.display import display, Markdown

display(Markdown(agent.render_profile()))


## 6A) Demo path A — Load an existing profile and generate a requirements package
This is the simplest workflow:
1. Load `demo/valve_profile_example.json`
2. Apply it to the agent
3. Generate the package (`/run` equivalent)


In [ ]:

# Reset and apply the example profile as a "patch"
agent.reset()
errs = agent.builder.apply_patch(agent.profile, example_profile)

print("Applied example profile. Errors:", errs)
print("Missing required:", agent.missing_required())

display(Markdown(agent.render_profile()))


In [ ]:

# Generate a package folder under demo/out/ and also a zip
pkg_dir = agent.run_and_package(package_name="demo_existing_profile")

print("Package dir:", pkg_dir)
print("Zip exists :", pkg_dir.with_suffix(".zip").exists())


### Inspect the manifest
`manifest.json` is meant as the **handoff contract** for downstream agents.
It provides a summary + file pointers.


In [ ]:

manifest_path = pkg_dir / "manifest.json"
manifest = json.loads(manifest_path.read_text(encoding="utf-8"))
manifest


## 6B) Demo path B — Elicit a profile iteratively
This section shows how the agent can build a profile from incremental user inputs.

Two modes:
- **LLM mode**: Provide free-text valve specs; the model extracts a schema-constrained JSON patch.
- **Manual mode**: If no LLM is configured, the agent will ask for missing required fields one by one, or you can use `agent.set_field()`.


In [ ]:

agent.reset()

print("LLM configured?", bool(llm))
print("Missing required fields:", agent.missing_required())

# Simulated user messages
messages = [
    "This is a safety-related MOV isolation valve on the primary loop. Tag is PLV-101. It is normally closed.",
    "Design pressure drop limit is 2.5 bar and max seat leakage is 0.1% rated flow.",
    "Stroke time limit 10 seconds. Materials: stainless steel body, Stellite hardfacing on seat.",
]

for m in messages:
    changed, response = agent.apply_user_text(m)
    print("\nUSER:", m)
    print("AGENT:", response)

display(Markdown(agent.render_profile()))
print("Missing required:", agent.missing_required())


### Manual filling helper
If you are in manual mode (no LLM), you can fill required fields explicitly:
```python
agent.set_field('valve_tag', 'PLV-101')
agent.set_field('actuation_type', 'MOV')
...
```
This notebook does not assume which fields are required; use the schema printout above.


## 7) Run requirements generation from the elicited profile
Once the profile is schema-complete, generate a package.
If required fields are still missing, you'll get a clear error.


In [ ]:

if agent.missing_required():
    print("Profile incomplete; missing:", agent.missing_required())
    print("Fill missing fields (LLM mode: provide more info; manual mode: agent.set_field(...))")
else:
    pkg_dir2 = agent.run_and_package(package_name="demo_elicited_profile")
    print("Package dir:", pkg_dir2)
    print("Zip exists :", pkg_dir2.with_suffix('.zip').exists())


## 8) Explore the requirements instance in a notebook-friendly view
We load `requirements_instance.json` and show:
- high-level summary
- a preview table of applicable requirements


In [ ]:

import pandas as pd

req_instance_path = pkg_dir / "requirements_instance.json"
req_instance = json.loads(req_instance_path.read_text(encoding="utf-8"))

req_instance["summary"], req_instance["validation"]["overall_status"]


In [ ]:

# Build a compact preview dataframe
rows = []
for r in req_instance["applicable_requirements"][:25]:
    rows.append({
        "id": r["id"],
        "status": r["status"],
        "verification": r["verification"]["method"],
        "tbd_parameters": ", ".join(r.get("tbd_parameters", [])),
        "text": r["text"],
    })

df = pd.DataFrame(rows)
df


## 9) Validation report review
The report is designed to be CI-friendly (aggregate counts by code and severity).


In [ ]:

report_csv = pkg_dir / "validation_report.csv"
report_df = pd.read_csv(report_csv)
report_df


## 10) Code-RAG demo (ground the assistant on *this repo*)
This is a *developer-facing* feature: ask questions like:
- How does `applicability.when` work?
- What validations are implemented?
- Where are reports generated?

If no LLM is configured, the agent will return the **top relevant code snippets**.


In [ ]:

question = "How does applicability.when work? What condition syntax is supported?"
print(agent.answer_dev_question(question))


## 11) Next enhancements (recommended)
Now that you have an interactive agent layer, the next major steps are:
1. **Requirements-RAG** over `data/primary_loop_valve_baseline_v0.2_150reqs.json`
   - answer: "show requirements about leak tightness" / "stroke time" etc.
2. **Standards clause database** + traceability mapping
   - add `trace[]` entries per requirement instance with (standard, edition, clause_id, evidence)
3. **Auto-fix pass** (optional)
   - split compound requirements flagged by atomicity warnings into atomic child requirements
